# New section

In [114]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [115]:
!cp /content/drive/My\ Drive/Flight_delay.ipynb /content/flight-delay-prediction/


cp: cannot stat '/content/drive/My Drive/Flight_delay.ipynb': No such file or directory


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import pandas as pd
import numpy as np


In [ ]:
#load Dataset
df = pd.read_csv('Generated_Flight_Data.csv')

In [ ]:
#Ensure that the columns that are categorical are encoded into numeric values
categorical_columns = ['Carrier', 'Origin', 'Destination', 'Weather']
encoder = OneHotEncoder(sparse_output=False)
encoded_columns = encoder.fit_transform(df[categorical_columns])
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

#Apply one-hot encoding to the catregorical colums
encoded_columns = encoder.fit_transform(df[categorical_columns])

In [ ]:
#convert encoded columns back to a DataFrame
encoded_categorical_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(categorical_columns))

In [ ]:
#Drop the original categorical columns from the original DataFrame
df_numeric = df.drop(columns = categorical_columns)

In [ ]:
#concatenate the encoded categorical columns with the rest of the numeric data
df_processed = pd.concat([df_numeric, encoded_categorical_df], axis=1)

In [ ]:

#Separate features (X) and target variable(y)
X = df_processed.drop(['Delay'], axis=1)
y = df_processed['Delay']

In [ ]:

#Split the data into training and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
print(X_train.columns)


Index(['FlightNumber', 'ScheduledDepTime', 'ActualDepTime', 'Carrier_AA',
       'Carrier_BA', 'Carrier_DL', 'Carrier_SW', 'Carrier_UA', 'Origin_ATL',
       'Origin_JFK', 'Origin_LAX', 'Origin_MIA', 'Origin_MSP',
       'Destination_ATL', 'Destination_DEN', 'Destination_LAX',
       'Destination_ORD', 'Destination_SFO', 'Weather_Clear', 'Weather_Fog',
       'Weather_Rain', 'Weather_Snow', 'Weather_Storm'],
      dtype='object')


In [ ]:
non_numeric_columns = X_train.select_dtypes(include=['object']).columns
print(non_numeric_columns)


Index(['ScheduledDepTime', 'ActualDepTime'], dtype='object')


In [ ]:
X_train['ScheduledDepTime'] = pd.to_datetime(X_train['ScheduledDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)
X_train['ActualDepTime'] = pd.to_datetime(X_train['ActualDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)
X_val['ScheduledDepTime'] = pd.to_datetime(X_val['ScheduledDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)
X_val['ActualDepTime'] = pd.to_datetime(X_val['ActualDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)
X_test['ScheduledDepTime'] = pd.to_datetime(X_test['ScheduledDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)
X_test['ActualDepTime'] = pd.to_datetime(X_test['ActualDepTime'], format='%H:%M').apply(lambda x: x.hour * 60 + x.minute)


In [ ]:
X_train = X_train.to_numpy().astype('float32')
y_train = y_train.to_numpy().astype('float32')
X_val = X_val.to_numpy().astype('float32')
y_val = y_val.to_numpy().astype('float32')
X_test = X_test.to_numpy().astype('float32')
y_test = y_test.to_numpy().astype('float32')

In [ ]:
#Check for the shapes of the datasets to ensure everything is correct
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (70, 23)
y_train shape: (70,)
X_val shape: (15, 23)
y_val shape: (15,)


In [ ]:
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(layers.InputLayer(shape=(23,)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='linear'))


optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
model.summary()




Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ (None, 128)                 │           3,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 32)                  │           1,056 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,553 (60.75 KB)

 Trainable params: 15,553 (60.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train type:", X_train.dtype)
print("y_train type:", y_train.dtype)


X_train shape: (70, 23)
y_train shape: (70,)
X_train type: float32
y_train type: float32


In [ ]:
#step 3: Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 122ms/step - loss: 4359.1362 - mae: 54.3712 - val_loss: 4503.5381 - val_mae: 62.9004
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 3533.7129 - mae: 49.1193 - val_loss: 3376.2163 - val_mae: 52.6517
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 2751.2236 - mae: 42.9325 - val_loss: 2497.5742 - val_mae: 42.7860
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2431.7991 - mae: 40.9983 - val_loss: 1859.0397 - val_mae: 34.3909
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2301.0388 - mae: 40.6699 - val_loss: 1471.9246 - val_mae: 27.5036
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2258.7563 - mae: 40.6372 - val_loss: 1254.5715 - val_mae: 25.5787
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2100.6479 - mae: 39.4148 - val_loss: 1135.1188 - val_mae: 24.7204
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2233.4387 - mae: 40.8912 - val_loss: 1061.3442 - val_mae: 24.0337
Epoch 9

In [ ]:
#step 4: Evaluate the model on the test set
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"Time delay error of around: {np.sqrt(mse)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Mean Squared Error: 956.9718017578125
Time delay error of around: 30.934961318969727


In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git clone https://github.com/JuneshG/flight-delay-prediction.git

Cloning into 'flight-delay-prediction'...


mv: cannot stat '/content/flight-delay-prediction/Generated_Flight_Data.csv': No such file or directory


In [ ]:
!mv /content/Generated_Flight_Data.csv /content/flight-delay-prediction/
!mv /content/Processed_Flight_Data.csv /content/flight-delay-prediction/


In [ ]:
!mv /content/Flight_delay.ipynb /content/flight-delay-prediction/


mv: cannot stat '/content/Flight_delay.ipynb': No such file or directory


In [ ]:
!ls /content


flight-delay-prediction  sample_data


In [ ]:
!find /content -name "*.ipynb"
